In [1]:
import os
import pandas as pd
import sys
sys.path.insert(0, '../data_prep')
from tmdbv3api import TMDb
from tmdbv3api import Movie, TV, Collection, Person
from sqlite_utils import get_from_table, delete_records, insert_record_into_table, df_to_table
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
tmdb_api_key = os.getenv('TMDB_API_KEY')
tmdb = TMDb()
tmdb.api_key = tmdb_api_key

In [4]:
film_id = 'f_00Msm'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
tmdb_id = get_from_table('TMDB_ID', film_id, 'TMDB_ID')
letterboxd_url = get_from_table('FILM_TITLE', film_id, 'LETTERBOXD_URL')
print('Film ID is "{}", film title is "{}", tmdb_id is "{}" and url is {}'.format(film_id, film_title, tmdb_id, letterboxd_url))

Film ID is "f_00Msm", film title is "Incendies", tmdb_id is "46738" and url is https://boxd.it/Msm


In [5]:
movie = Movie()
details = movie.details(tmdb_id)
details

{'adult': False, 'backdrop_path': '/aFmoJntgDPR0Jd8XD1ZfFSmcqMg.jpg', 'belongs_to_collection': None, 'budget': 6800000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 10752, 'name': 'War'}, {'id': 9648, 'name': 'Mystery'}], 'homepage': 'http://www.sonyclassics.com/incendies/', 'id': 46738, 'imdb_id': 'tt1255953', 'origin_country': ['CA'], 'original_language': 'fr', 'original_title': 'Incendies', 'overview': "A mother's last wishes send twins Jeanne and Simon on a journey to Middle East in search of their tangled roots. Adapted from Wajdi Mouawad's acclaimed play, Incendies tells the powerful and moving tale of two young adults' voyage to the core of deep-rooted hatred, never-ending wars and enduring love.", 'popularity': 21.2, 'poster_path': '/47Y5xxjNef07h2EiyBJNIX3eoQP.jpg', 'production_companies': [{'id': 313, 'logo_path': '/rx5CT9wUnEz0AcdpcX9epujv7nw.png', 'name': 'TS Productions', 'origin_country': 'FR'}, {'id': 7273, 'logo_path': None, 'name': 'Phi Group', 'origin_country': ''},

In [48]:
crew = [
    'Director',
    'Director of Photography',
    'Original Music Composer',
    'Screenplay',
    'Writer'
]

In [53]:
[x for x in details['casts']['crew'] if x['job'] in crew]

[{'adult': False, 'gender': 2, 'id': 2950, 'known_for_department': 'Camera', 'name': 'Robert Elswit', 'original_name': 'Robert Elswit', 'popularity': 3.042, 'profile_path': '/5urU681Z1ZyJxfjEGp6qY6XYXg1.jpg', 'credit_id': '52fe447bc3a36847f8098a33', 'department': 'Camera', 'job': 'Director of Photography'},
 {'adult': False, 'gender': 2, 'id': 4762, 'known_for_department': 'Directing', 'name': 'Paul Thomas Anderson', 'original_name': 'Paul Thomas Anderson', 'popularity': 7.012, 'profile_path': '/29DHpRKddqTwo2NU162Y49JoQYa.jpg', 'credit_id': '52fe447bc3a36847f8098a23', 'department': 'Writing', 'job': 'Screenplay'},
 {'adult': False, 'gender': 2, 'id': 4762, 'known_for_department': 'Directing', 'name': 'Paul Thomas Anderson', 'original_name': 'Paul Thomas Anderson', 'popularity': 7.012, 'profile_path': '/29DHpRKddqTwo2NU162Y49JoQYa.jpg', 'credit_id': '52fe447bc3a36847f8098a5d', 'department': 'Directing', 'job': 'Director'},
 {'adult': False, 'gender': 2, 'id': 74333, 'known_for_departme

In [57]:
person = Person()

In [58]:
person_test = person.details('138')

In [60]:
person_test.keys()

dict_keys(['adult', 'also_known_as', 'biography', 'birthday', 'deathday', 'gender', 'homepage', 'id', 'imdb_id', 'known_for_department', 'name', 'place_of_birth', 'popularity', 'profile_path', 'images'])

In [69]:
person_attrs = [
    'id',
    'name',
    'imdb_id',
    'gender',
    'birthday',
    'deathday',
    'known_for_department',
    'place_of_birth',
    'popularity'
]

In [70]:
for attr in person_attrs:
    print(attr, ' = ', person_test.get(attr))

id  =  138
name  =  Quentin Tarantino
imdb_id  =  nm0000233
gender  =  2
birthday  =  1963-03-27
deathday  =  None
known_for_department  =  Directing
place_of_birth  =  Knoxville, Tennessee, USA
popularity  =  20.438


In [3]:
attrs = {
    'belongs_to_collection': None,
    'budget': None,
    'original_language': 'languages',
    'popularity': 'popularity',
    'release_date': 'first_air_date',
    'revenue': None,
    'runtime': ['episode_run_time', 'number_of_episodes'],
    'status': 'status',
    'vote_average': 'vote_average',
    'vote_count': 'vote_count',
    'keywords': 'keywords',
    'casts': 'credits'
}

In [4]:
def create_movie_metadata_dict(film_id):
    film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
    tmdb_id = get_from_table('TMDB_ID', film_id, 'TMDB_ID')
    content_type = get_from_table('CONTENT_TYPE', film_id, 'CONTENT_TYPE')
    print('Film ID is "{}", title is "{}", and TMDB id is "{}", while content type is "{}"'.format(film_id, film_title, tmdb_id, content_type))
    movie_metadata_dict = {'FILM_ID': film_id}
    if content_type == 'movie':
        movie = Movie()
        try:
            details = movie.details(tmdb_id)
            for k in attrs:
                movie_metadata_dict[k] = details.get(k, None)
        except:
            None
    elif content_type == 'tv':
        tv = TV()
        details = tv.details(tmdb_id, 'credits,keywords')
        for k in attrs:
            if type(attrs[k]) == list:
                movie_metadata_dict[k] = [details.get(x, None) for x in attrs[k]]
            else:
                movie_metadata_dict[k] = details.get(attrs[k], None)
        movie_metadata_dict['original_language'] = movie_metadata_dict['original_language'][0]
        movie_metadata_dict['runtime'] = movie_metadata_dict['runtime'][0][0] * movie_metadata_dict['runtime'][1]
        movie_metadata_dict['status'] = movie_metadata_dict['status'].replace('Returning Series', 'Released').replace('Ended', 'Released')
        if len(movie_metadata_dict.get('keywords', {'keywords': []}).get('keywords')) == 0:
               movie_metadata_dict['keywords']['keywords'] = [{'id': -1, 'name': 'none'}]
    else:
        details = {}
    return {'film_title': film_title, 'tmdb_id': tmdb_id, 'content_type':content_type, 'movie_metadata_dict': movie_metadata_dict, 'details': details}

In [5]:
f_097fO = create_movie_metadata_dict('f_097fO')
f_097fO

Film ID is "f_097fO", title is "Wyrmwood: Road of the Dead", and TMDB id is "290999", while content type is "movie"


{'film_title': 'Wyrmwood: Road of the Dead',
 'tmdb_id': '290999',
 'content_type': 'movie',
 'movie_metadata_dict': {'FILM_ID': 'f_097fO',
  'belongs_to_collection': {'id': 927812, 'name': 'Wyrmwood Collection', 'poster_path': '/tRSNW8dte5P9L8vZPaCFFkjAOnI.jpg', 'backdrop_path': '/drstxfAEQ8tR1qf9Axzg6ZnhGxn.jpg'},
  'budget': 0,
  'original_language': 'en',
  'popularity': 11.539,
  'release_date': '2014-09-19',
  'revenue': 0,
  'runtime': 98,
  'status': 'Released',
  'vote_average': 6.089,
  'vote_count': 384,
  'keywords': {'keywords': [{'id': 12377, 'name': 'zombie'}, {'id': 186565, 'name': 'zombie apocalypse'}]},
  'casts': {'cast': [{'adult': False, 'gender': 0, 'id': 1262388, 'known_for_department': 'Acting', 'name': 'Jay Gallagher', 'original_name': 'Jay Gallagher', 'popularity': 1.757, 'profile_path': '/ppwekPN23qcJER0Wiih5sQi1JwW.jpg', 'cast_id': 1, 'character': 'Barry', 'credit_id': '540bbc1dc3a368799f0027f7', 'order': 0}, {'adult': False, 'gender': 1, 'id': 1326440, 'kno

In [9]:
f_097fO['details']

{'adult': False, 'backdrop_path': '/k1KP47cnuscBPS4yYTJJ6S4PDho.jpg', 'belongs_to_collection': {'id': 927812, 'name': 'Wyrmwood Collection', 'poster_path': '/tRSNW8dte5P9L8vZPaCFFkjAOnI.jpg', 'backdrop_path': '/drstxfAEQ8tR1qf9Axzg6ZnhGxn.jpg'}, 'budget': 0, 'genres': [{'id': 27, 'name': 'Horror'}, {'id': 28, 'name': 'Action'}, {'id': 35, 'name': 'Comedy'}], 'homepage': '', 'id': 290999, 'imdb_id': 'tt2535470', 'original_language': 'en', 'original_title': 'Wyrmwood: Road of the Dead', 'overview': 'Barry is a talented mechanic and family man whose life is torn apart on the eve of a zombie apocalypse. His sister, Brooke, is kidnapped by a sinister team of gas-mask wearing soldiers & experimented on by a psychotic doctor. While Brooke plans her escape Barry goes out on the road to find her & teams up with Benny, a fellow survivor - together they must arm themselves and prepare to battle their way through hordes of flesh-eating monsters in a harsh Australian bushland.', 'popularity': 11.53

,COLLECTION_ID,TMDB_ID,FILM_TITLE
0,927812,290999,Wyrmwood: Road of the Dead
1,927812,879957,Wyrmwood: Apocalypse


In [145]:
keywords = f_0lrky[3].get('keywords', {'keywords': [{'id': -1, 'name': 'none'}]})
keywords


KeyError: 3

In [ ]:
if keywords:
    keywords = keywords.get('keywords')
    keyword_df = pd.DataFrame({
        'FILM_ID': [film_id]*len(keywords),
        'KEYWORD_ID':[x.get('id') for x in keywords],
        'KEYWORD':[x.get('name') for x in keywords],
        'CREATED_AT': [datetime.now()]*len(keywords)
                                })

In [5]:
tv = TV()
details = tv.details('87108', 'credits,keywords')

In [11]:
for i in details:
    print(i, ':', details[i])

adult : False
backdrop_path : /20eIP9o5ebArmu2HxJutaBjhLf4.jpg
created_by : [{'id': 35796, 'credit_id': '5ca7a5990e0a264c69f04ea8', 'name': 'Craig Mazin', 'gender': 2, 'profile_path': '/uEhna6qcMuyU5TP7irpTUZ2ZsZc.jpg'}]
episode_run_time : []
first_air_date : 2019-05-06
genres : [{'id': 18, 'name': 'Drama'}]
homepage : https://www.hbo.com/chernobyl
id : 87108
in_production : False
languages : ['en']
last_air_date : 2019-06-03
last_episode_to_air : {'id': 1747127, 'name': 'Vichnaya Pamyat', 'overview': 'Valery Legasov, Boris Shcherbina and Ulana Khomyuk risk their lives and reputations to expose the truth about Chernobyl.', 'vote_average': 9.297, 'vote_count': 158, 'air_date': '2019-06-03', 'episode_number': 5, 'episode_type': 'finale', 'production_code': '', 'runtime': 73, 'season_number': 1, 'show_id': 87108, 'still_path': '/mgXm2Y6SOFDHtD5thi2LmS7uQBj.jpg'}
name : Chernobyl
next_episode_to_air : None
networks : [{'id': 49, 'logo_path': '/tuomPhY2UtuPTqqFnKMVHvSb724.png', 'name': 'HBO

In [87]:
f_01UTC['details']['original_language']

'en'

In [88]:
for i in f_01UTC['details']:
    print(i)

adult
backdrop_path
belongs_to_collection
budget
genres
homepage
id
imdb_id
original_language
original_title
overview
popularity
poster_path
production_companies
production_countries
release_date
revenue
runtime
spoken_languages
status
tagline
title
video
vote_average
vote_count
videos
trailers
images
casts
translations
keywords
release_dates


In [89]:
f_01U58['details']['languages']

['en']

In [118]:
f_01U58_2 = create_movie_metadata_dict('f_01U58')

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"


In [119]:
f_01U58_2['movie_metadata_dict']#['original_language']

{'FILM_ID': 'f_01U58',
 'belongs_to_collection': None,
 'budget': None,
 'original_language': 'en',
 'popularity': 20.037,
 'release_date': '1979-11-17',
 'revenue': None,
 'runtime': 184,
 'status': 'Ended',
 'vote_average': 6.9,
 'vote_count': 383,
 'keywords': {'results': [{'name': 'based on novel or book', 'id': 818}, {'name': 'vampire', 'id': 3133}, {'name': 'home', 'id': 1526}, {'name': 'return', 'id': 1571}, {'name': 'menace', 'id': 2250}, {'name': 'antiquary', 'id': 3095}, {'name': 'house', 'id': 6095}, {'name': 'miniseries', 'id': 11162}, {'name': 'writer', 'id': 13028}, {'name': 'new england', 'id': 41393}, {'name': 'hometown', 'id': 155808}, {'name': 'horror', 'id': 315058}]},
 'casts': {'cast': [{'adult': False, 'gender': 2, 'id': 15956, 'known_for_department': 'Acting', 'name': 'David Soul', 'original_name': 'David Soul', 'popularity': 8.281, 'profile_path': '/y7fW77RzeqgBYeoTswsMfwy8FwI.jpg', 'character': 'Ben Mears', 'credit_id': '52584c96760ee34661008d4c', 'order': 0}, 

In [90]:
f_01U58 = create_movie_metadata_dict('f_01U58')
for i in f_01U58['details']:
    print(i)

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"
adult
backdrop_path
created_by
episode_run_time
first_air_date
genres
homepage
id
in_production
languages
last_air_date
last_episode_to_air
name
next_episode_to_air
networks
number_of_episodes
number_of_seasons
origin_country
original_language
original_name
overview
popularity
poster_path
production_companies
production_countries
seasons
spoken_languages
status
tagline
type
vote_average
vote_count
credits
keywords


In [72]:
f_01U58['details']['episode_run_time'][0] * f_01U58['details']['number_of_episodes']

184

In [91]:
f_01U58['details']['keywords']

{'results': [{'name': 'based on novel or book', 'id': 818}, {'name': 'vampire', 'id': 3133}, {'name': 'home', 'id': 1526}, {'name': 'return', 'id': 1571}, {'name': 'menace', 'id': 2250}, {'name': 'antiquary', 'id': 3095}, {'name': 'house', 'id': 6095}, {'name': 'miniseries', 'id': 11162}, {'name': 'writer', 'id': 13028}, {'name': 'new england', 'id': 41393}, {'name': 'hometown', 'id': 155808}, {'name': 'horror', 'id': 315058}]}

In [48]:
film_id = 'f_01U58'
film_title = get_from_table('FILM_TITLE', film_id, 'FILM_TITLE')
tmdb_id = get_from_table('TMDB_ID', film_id, 'TMDB_ID')
content_type = get_from_table('CONTENT_TYPE', film_id, 'CONTENT_TYPE')
print('Film ID is "{}", title is "{}", and TMDB id is "{}", while content type is "{}"'.format(film_id, film_title, tmdb_id, content_type))

Film ID is "f_01U58", title is "Salem's Lot", and TMDB id is "16118", while content type is "tv"


In [92]:
tv = TV()
details = tv.details('f_01U58', 'credits,keywords,budget')

TMDbException: The resource you requested could not be found.

In [161]:
from justwatch import JustWatch

just_watch = JustWatch(country='GB')


In [ ]:
total_pages = just_watch.search_for_item(providers=['nfx'], content_types=['movie']).get('total_pages')

In [164]:
just_watch.search_for_item(providers=['nfx'], content_types=['movie']).get('items')

[{'jw_entity_id': 'tm860276',
  'id': 860276,
  'title': 'Nobody',
  'full_path': '/uk/movie/nobody',
  'full_paths': {'MOVIE_DETAIL_OVERVIEW': '/uk/movie/nobody'},
  'poster': '/poster/239506757/{profile}',
  'poster_blur_hash': 'TE9?:,WW0errayI;tRofni9aof-o',
  'object_type': 'movie',
  'scoring': [{'provider_type': 'jw:popular_1_day', 'value': 377.67},
   {'provider_type': 'imdb:votes', 'value': 280493},
   {'provider_type': 'jw:popular_180_day', 'value': 412.99},
   {'provider_type': 'jw:viral', 'value': 0.07},
   {'provider_type': 'imdb:score', 'value': 7.4},
   {'provider_type': 'tmdb:votes', 'value': 5813},
   {'provider_type': 'tmdb:id', 'value': 615457},
   {'provider_type': 'imdb:multiplied', 'value': 2075648.2000000002},
   {'provider_type': 'tmdb:score', 'value': 8.055},
   {'provider_type': 'tmdb:popularity', 'value': 83.242},
   {'provider_type': 'imdb:popularity', 'value': 35},
   {'provider_type': 'jw:popular_7_day', 'value': 478.3},
   {'provider_type': 'jw:popular_30_